In [ ]:
from PosSelect_Functions_Old import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pandas as pd
import numpy as np
import copy
import seaborn as sns
from scipy.stats import mannwhitneyu as mwu
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import wilcoxon
from scipy.optimize import curve_fit
from scipy.stats import fisher_exact

hfont = {'fontname':'Arial'}
plt.rcParams["font.family"] = "Arial"

#Code borrowed heavily from here: https://stackoverflow.com/questions/62375034/find-non-overlapping-area-between-two-kde-plots
plt.rcParams.update(
    {"text.usetex": False}
)

#Define a logarithmic function to fit to the data
def plot_stuff(df, title, xlabel, ylabel):
    def func(x, a, c, d):
        return a + d*np.log(x + c)

    #Fit said function
    popt, pcov = curve_fit(func, xdata=df["AF bin"], ydata=df["Alpha"], maxfev = 100000)
    sns.scatterplot(x=df["AF bin"], y=df["Alpha"])
    xx = np.linspace(0.1, 1, 100)
    yy = func(xx, *popt)
    plt.title(title, size = 18)
    plt.xlabel(xlabel, size = 15)
    plt.ylabel(ylabel, size = 15)
    plt.plot(xx, yy)

gobp = pd.read_csv("../DPSC_CNCC/GOBP_AccelEvol_Input.txt", sep= "\t")
d_BP = {}

for index, row in gobp.iterrows():
    if "yloid" in row["Term"]:
        print(row["Term"])
    d_BP[row["Term"]] = row["Genes"].split(";")


In [ ]:
#Compute alpha overall
v, yvalls = read_missense(path = "./", maf_cut = 0.1)
yvalls = unfold_missense(yvalls)

yvals2 = [np.float64(j) for j in list(yvalls["PhyloP447"])]
yvals2.sort()

cuttt = 0.6
cutoff = yvals2[int(floor((len(yvals2)*cuttt)))]

vvv = prepare_alpha(v, yvalls)
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, window = [-5, 12])

In [ ]:
#Asymptotic for viral proteins
v, yvalls = read_missense(path = "./", maf_cut = 0.1)
yvalls = unfold_missense(yvalls)

#Read in the list of VIPs
vip = pd.read_csv("GeneSets/VIP.csv")

v_vip = v[v["Gene"].isin(vip["HGNC symbol"])]
yvalls_vip = yvalls[yvalls["Gene"].isin(vip["HGNC symbol"])]
v_nvip = v[~v["Gene"].isin(vip["HGNC symbol"])]
yvalls_nvip = yvalls[~yvalls["Gene"].isin(vip["HGNC symbol"])]

alpha, to_plot = asymptotic_unfold_cutoff(v_vip, yvalls_vip, dn_cut = 0.05, start = 0.1, cuttt = 0.6, to_plot_curve = False)
print(alpha)
plot_stuff(to_plot, title = "Stronger evidence for positive\nselection on viral-interacting proteins", ylabel = "$\\alpha_{Cons}$", xlabel = "Derived allele frequency bin")

In [ ]:
#Read in data
v_mis, vv_mis = read_missense(path = "./", maf_cut = 0.1)
vv_mis = unfold_missense(vv_mis)



In [ ]:
#Restrict to VIPs
vip = pd.read_csv("GeneSets/VIP.csv")

v_mis2 = v_mis[v_mis["Gene"].isin(vip["HGNC symbol"])]
vv_mis2 = vv_mis[vv_mis["Gene"].isin(vip["HGNC symbol"])]


v_mis2[v_mis2["PhyloP447"] < -2].sort_values("PhyloP447").head(50)

In [ ]:
#Test for enrichment in negative PhyloP genes
neg = v_mis2[v_mis2["PhyloP447"] <= -3].shape[0]
pos = v_mis2[v_mis2["PhyloP447"] >= 0].shape[0]
out = []
c = 0
for key in d_BP.keys():
    c += 1
    genes = d_BP[key]
    if len(genes) >= 15:
        vmb = v_mis2[v_mis2["Gene"].isin(genes)]
        neg_in = vmb[vmb["PhyloP447"] <= -3].shape[0]
        pos_in = vmb[vmb["PhyloP447"] >= 0].shape[0]
        if neg_in + pos_in >= 10:
            neg_out = neg - neg_in
            pos_out = pos - pos_in

            table = [[neg_in, neg_out], [pos_in, pos_out]]
            out.append([key, fisher_exact(table)[0], fisher_exact(table)[1], neg_in, neg_out])
    if c % 1000 == 0:
        print(c)

In [ ]:
df = pd.DataFrame(out)
df["FDR"] = fdrcorrection(df[2])[1]
df = df.sort_values("FDR")
df.columns = ["Gene set", "Odds ratio", "p-value", "X", "Y", "FDR"]
df.head(50)

In [ ]:
#Make volcano plot
dff = df.copy()
dff["-Log$_{10}$(FDR)"] = -np.log10(dff["FDR"])

k = []
for index, row in dff.iterrows():
    if row["FDR"] < 0.05:
        k.append("FDR < 0.05")
    else:
        k.append("Not significant")
dff['Significance'] = k
fig, ax = plt.subplots(figsize=(10,6))
sns.scatterplot(data = dff, x = "Odds ratio", y = "-Log$_{10}$(FDR)", hue = "Significance", palette = {"FDR < 0.05":"red", "Not significant":"grey"})
plt.title("Biological process enrichments for negative PhyloP sites", size = 18)
plt.ylabel("-Log$_{10}$(FDR)", size = 18)
plt.xlabel("Odds ratio", size = 18)
#plt.xticks(size = 11)
#plt.yticks(size = 11)
plt.legend(fontsize = 12)


In [ ]:
vs, vvs = read_syn(maf_cut = 0.1)
vvs = unfold_syn(vvs)
vvs2 = vvs[(vvs["UnfoldedMAF"] > 0.5) & (vvs["UnfoldedMAF"] < 0.9)]

In [ ]:
vk = v_mis2[v_mis2["Gene"].isin(np.intersect1d(d_BP["Phagocytosis (GO:0006909)"], vip["HGNC symbol"]))]
vvk = vv_mis3[vv_mis3["Gene"].isin(np.intersect1d(d_BP["Phagocytosis (GO:0006909)"], vip["HGNC symbol"]))]

vsk = vvs[vvs["Gene"].isin(np.intersect1d(d_BP["Phagocytosis (GO:0006909)"], vip["HGNC symbol"]))]
vvsk = vvs2[vvs2["Gene"].isin(np.intersect1d(d_BP["Phagocytosis (GO:0006909)"], vip["HGNC symbol"]))]

vvk = vvk[(vvk["UnfoldedMAF"] > 0.5) & (vvk["UnfoldedMAF"] < 0.9)]
vvsk = vvsk[(vvsk["UnfoldedMAF"] > 0.5) & (vvsk["UnfoldedMAF"] < 0.9)]

fisher_exact([[vk.shape[0], vvk.shape[0]], [vsk.shape[0], vvsk.shape[0]]], alternative = "greater")

In [ ]:
#Doing it for all genes genome-wide
neg = v_mis[v_mis["PhyloP447"] <= -3].shape[0]
pos = v_mis[v_mis["PhyloP447"] >= 0].shape[0]
out = []
c = 0
for key in d_BP.keys():
    c += 1
    genes = d_BP[key]
    if len(genes) >= 15:
        vmb = v_mis[v_mis["Gene"].isin(genes)]
        neg_in = vmb[vmb["PhyloP447"] <= -3].shape[0]
        pos_in = vmb[vmb["PhyloP447"] >= 0].shape[0]
        if neg_in + pos_in >= 10:
            neg_out = neg - neg_in
            pos_out = pos - pos_in

            table = [[neg_in, neg_out], [pos_in, pos_out]]
            out.append([key, fisher_exact(table)[0], fisher_exact(table)[1], neg_in, neg_out])
    if c % 1000 == 0:
        print(c)

In [ ]:
df = pd.DataFrame(out)
df["FDR"] = fdrcorrection(df[2])[1]
df = df.sort_values("FDR")
df.columns = ["Gene set", "Odds ratio", "p-value", "X", "Y", "FDR"]
df.head(50)

In [ ]:
dff = df.copy()
dff["-Log$_{10}$(FDR)"] = -np.log10(dff["FDR"])

k = []
for index, row in dff.iterrows():
    if row["FDR"] < 0.05:
        k.append("FDR < 0.05")
    else:
        k.append("Not significant")
dff['Significance'] = k
fig, ax = plt.subplots(figsize=(10,6))
sns.scatterplot(data = dff, x = "Odds ratio", y = "-Log$_{10}$(FDR)", hue = "Significance", palette = {"FDR < 0.05":"red", "Not significant":"grey"})
plt.title("Biological process enrichments for negative PhyloP sites, all genes", size = 18)
plt.ylabel("-Log$_{10}$(FDR)", size = 18)
plt.xlabel("Odds ratio", size = 18)
#plt.xticks(size = 11)
#plt.yticks(size = 11)
plt.legend(fontsize = 12)


In [ ]:
vv_mis3 = vv_mis[(vv_mis["UnfoldedMAF"] > 0.5) & (vv_mis2["UnfoldedMAF"] < 0.9)]
vvs2 = vvs[(vvs["UnfoldedMAF"] > 0.5) & (vvs["UnfoldedMAF"] < 0.9)]

vk = v_mis2[v_mis2["Gene"].isin(genes)].shape[0]
vvk = vv_mis3[vv_mis3["Gene"].isin(genes)].shape[0]

vsk = vvs[vvs["Gene"].isin(genes)].shape[0]
vvsk = vvs2[vvs2["Gene"].isin(genes)].shape[0]

fisher_exact([[vk, vvk], [vsk, vvsk]], alternative = "greater")

In [ ]:
#Plotting VIP distributions
vip = pd.read_csv("GeneSets/VIP.csv")

v_mis2 = v_mis[v_mis["Gene"].isin(vip["HGNC symbol"])]
vv_mis2 = vv_mis[vv_mis["Gene"].isin(vip["HGNC symbol"])]

vv_mis3 = vv_mis2[(vv_mis2["UnfoldedMAF"] > 0.5) & (vv_mis2["UnfoldedMAF"] < 0.9)]

#v_mis2 = v_mis2[v_mis2["PhyloP447"] > -2]
#vv_mis3 = vv_mis3[vv_mis3["PhyloP447"] > -2]

v_mis2 = v_mis2[v_mis2["PhyloP447"] < 2]
vv_mis3 = vv_mis3[vv_mis3["PhyloP447"] < 2]

#v_mis2["PhyloP447"] = -v_mis2["PhyloP447"]
#vv_mis3["PhyloP447"] = -vv_mis3["PhyloP447"]

yvals2 = [np.float64(j) for j in list(vv_mis3["PhyloP447"])]
yvals2.sort()

cuttt = 0.6
cutoff = yvals2[int(floor((len(yvals2)*cuttt)))]

vvv = prepare_alpha(v_mis2, vv_mis3)
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, window = [-10, 12])
print(alpha)
print((fisher_exact(alpha[-3], alternative = "greater")[1] + fisher_exact(alpha[-2], alternative = "greater")[1])/2)
plt.title("Distribution of PhyloP scores > -2 for viral-interacting proteins", size = 22)


In [ ]:
#Doing analysis for solvent accessibility
v, yvalls = read_missense(path = "./", maf_cut = 0.1)
yvalls = unfold_missense(yvalls)

vx = pd.read_csv("ProteinLevel/ProtVar_HC_Dedup.sort.prot.bed", sep = "\t", header = None).set_index(3)
v["PositionPlus"] = v.index
v.index = v["Position"]
vu = v.join(vx).dropna()
v.index = v["PositionPlus"]
vu.index = vu["Position"]

vvx = pd.read_csv("ProteinLevel/ProtVar_Poly_Dedup.sort.prot.bed", sep = "\t", header = None).set_index(3)
yvalls["PositionPlus"] = yvalls.index
yvalls.index = yvalls["Position"]
yvallsu = yvalls.join(vvx).dropna()
yvalls.index = yvalls["PositionPlus"]
yvallsu.index = yvallsu["Position"]

v = v.drop_duplicates("Position")
yvalls = yvalls.drop_duplicates("Position")

vu = vu.drop_duplicates("Position")
yvallsu = yvallsu.drop_duplicates("Position")

In [ ]:
pse = pd.read_csv("ProteinLevel/20230315_Supplementary_AlphaFold_pPSE.csv")
pse

In [ ]:
pse = pd.read_csv("ProteinLevel/20230315_Supplementary_AlphaFold_pPSE.csv")
pse.index = pse["protein_id"].astype(str) + ":" + pse["position"].astype(str)
vu.index = vu[0].astype(str) + ":" + vu[1].astype(int).astype(str)
yvallsu.index = yvallsu[0].astype(str) + ":" + yvallsu[1].astype(int).astype(str)
vu_pse = vu.join(pse).dropna()
yvallsu_pse = yvallsu.join(pse).dropna()
yvalls

In [ ]:
yvallsu_pse_high = yvallsu_pse[yvallsu_pse["pPSE"] >= 5]
yvallsu_pse_low = yvallsu_pse[yvallsu_pse["pPSE"] < 1]

vu_pse_high = vu_pse[vu_pse["pPSE"] >= 5]
vu_pse_low = vu_pse[vu_pse["pPSE"] < 1]

In [ ]:
yvallsu_pse_low = yvallsu_pse_low[(yvallsu_pse_low["UnfoldedMAF"] > 0.5) & (yvallsu_pse_low["UnfoldedMAF"] < 0.9)]
yvallsu_pse_high = yvallsu_pse_high[(yvallsu_pse_high["UnfoldedMAF"] > 0.5) & (yvallsu_pse_high["UnfoldedMAF"] < 0.9)]

In [ ]:
vvv = prepare_alpha(vu_pse_low, yvallsu_pse_low)
cuttt = 0.6
z = list(yvallsu_pse_high["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]

compute_alpha_cutoff(vvv, cutoff = cutoff, plot = True, title = "High sidechain accessibility", window = [-5, 12])

In [ ]:
#Confidence interval or MAF 0.1-0.9
b = []
for i in range(1000):
    if i % 100 == 0:
        print(i)
    vvv = prepare_alpha(vu_pse_high.sample(frac = 1, replace = True), yvallsu_pse_high.sample(frac = 1, replace = True))
    cuttt = 0.6
    z = list(yvallsu_pse_high["PhyloP447"])
    z.sort()
    cutoff = z[int(floor((len(z)*cuttt)))]

    b.append(compute_alpha_cutoff(vvv, cutoff = cutoff, plot = False, title = "Low sidechain accessibility", window = [-5, 12])[0])

In [ ]:
b.sort()
print(b[25], b[975])